# HackHPC-20-Min-Neighborhoods

## Description:
  This program is an evalutation tool in Python using Googlemaps python library to find the places within a 20 minute walk or bike ride. These places include hospitals, pharmacies, grocery stores, all basic need providers. This tool can be used to determine if a neighborhood is accessible by determining how many of these places are available in a 20 min range. 
 
## More Info:
This project is part of HackHPC - SC22's HPC in the City: Dallas hackathon

   - [SC22's HPC in the City: Dallas - Page](https://hackhpc.github.io/HPCintheCity22/)
    
   - [Github](https://github.com/Winter-1314/HackHPC-20-Min-Neighborhoods)
   
 # Program:
 Install all the libraries required.

In [ ]:
!pip install googlemaps
!pip install gmaps

Defaulting to user installation because normal site-packages is not writeable


### Define Global Var

In [5]:
RADIUS = [10000, 1000, 1600, 2000]
ACTIONS = ["Y", "N"]
MODE_OF_TRANS = ["driving", "walking", "bicycling", "transit"]
FIND_PLACE_TEXTQUERY = "textquery"
FIND_PLACE_PHONENUMBER = "phonenumber"
PLACE_ID = "place_id"
ADDRESS_COMMA = ["locality", "route"]
ADDRESS_NEWLINE = ["point_of_interest", "establishment"]
ADDRESS_IGNORE = ["administrative_area_level_2", "administrative_area_level_3"]

### Setup API with APU Key For Use

In [6]:
with open('key.txt') as f:
    api_key = f.readline()
    f.close
gomaps = googlemaps.Client(key=api_key)

NameError: name 'googlemaps' is not defined

## Define all user interaction functions:

In [ ]:
def collectInfo() -> str:
    """
    Collects the address of the location that the user is looking for.
    :return: The address of the location the user entered
    :rtype: str
    """
    # First Collect Address From User - Input
    num_of_attept = 0
    correct_location = False
    while correct_location != True:
        list_of_locations = []
        if num_of_attept > 0:
            print("Please enter a more detailed address.")
        search_place = input(
            "Please enter a name, address, or phone number as your starting location:\n"
        )
        origin_location = gomaps.find_place(input=search_place,
                                            input_type=FIND_PLACE_TEXTQUERY)
        origin_location_status = origin_location.get("status")
        while (origin_location_status != "OK"):
            search_place = input(
                "Invaild Location!\nPlease enter a name, address, or phone number as your starting location:\n"
            )
            origin_location = gomaps.find_place(
                input=search_place, input_type=FIND_PLACE_TEXTQUERY)
            origin_location_status = origin_location.get("status")
        origin_location_place_id = origin_location.get("candidates")
        for places in origin_location_place_id:
            location_data = gomaps.place(place_id=places.get(PLACE_ID))
            location_address = location_data.get("result").get(
                "address_components")
            address_string = ""
            for parts in location_address:
                type_address = parts.get("types")[0]
                if type_address in ADDRESS_IGNORE:
                    continue
                elif type_address in ADDRESS_NEWLINE:
                    address_string = address_string + " " + parts.get(
                        "long_name") + "\n"
                elif type_address in ADDRESS_COMMA:
                    address_string = address_string + " " + parts.get(
                        "long_name") + ","
                else:
                    address_string = address_string + " " + parts.get(
                        "long_name")
            list_of_locations.append(address_string)
        temp = input(
            f"Is this address below your address?(Y/N):\n{list_of_locations[0]}\n"
        )
        while temp.upper() not in ACTIONS:
            temp = input("Please enter Y for Yes and N for no (Y/N):\n")
        if temp.upper() == "Y":
            correct_location = True
        num_of_attept += 1
    return list_of_locations[0]


def collectTransportation() -> int:
    """
    Collects the type of transportion that the data is going to be calculated for
    :return: The number that correspondes to the type of transportation
    :rtype: int
    """
    # Secondly, Collects Transportation Method From User - Input
    type_of_trans = input(
        f"What type of transportation do you want to use?\n\t1.{MODE_OF_TRANS[0]}\n\t2.{MODE_OF_TRANS[1]}\n\t3.{MODE_OF_TRANS[2]}\n\t4.{MODE_OF_TRANS[3]}\nPlease enter a number between 1-4:\n"
    )
    while int(type_of_trans) not in [1, 2, 3, 4]:
        print("Please make sure you enter a number is and that is between 1-4")
        type_of_trans = input(
            f"What type of transportation do you want to use?\n\t1.{MODE_OF_TRANS[0]}\n\t2.{MODE_OF_TRANS[1]}\n\t3.{MODE_OF_TRANS[2]}\n\t4.{MODE_OF_TRANS[3]}\nPlease enter a number between 1-4:\n"
        )
    return int(type_of_trans) - 1

## Define all computational functions:

In [ ]:

def getGeoCode(address: str) -> list:
    """
    Given address get GeoCode that includes the lat and long version of that address
    :param address: The address we want to get the GeoCode
    :type address: str
    :return: The long and lat in list format
    :rtype: list
    """
    # Convert User Collected Address to Lat and Long - geocode()
    location_geocode = gomaps.geocode(address=address)
    lat = location_geocode[0].get("geometry").get("location").get("lat")
    long = location_geocode[0].get("geometry").get("location").get("lng")
    lat_long_lst = [lat, long]
    return lat_long_lst


def serechForPlaces(lat_long: list, radius: int, type: str = None) -> dict:
    """
    Computes all the locations that are near the address provided as a lat/long
    :param lat_long: List version of the lat and long
    :type lat_long: list
    :param radius: The radius that we want to search from the lat and long
    :type radius: int
    :param type: Any specific type of location we want to look for
    :type type: str
    :return: A dictionary with all the locations nearby
    :rtype: dict
    """
    # Serech for places near an address in a radius and type - places_nearby()
    location_dict = {}
    if type == None:
        search_info = gomaps.places_nearby(location=lat_long, radius=radius)
    else:
        search_info = gomaps.places_nearby(location=lat_long,
                                           radius=radius,
                                           type=type)
    search_result = search_info.get("results")
    for index in search_result:
        location_dict[index.get("name")] = {}
        location_dict[index.get("name")]["status"] = index.get(
            "business_status")
        location_dict[index.get("name")]["place_id"] = index.get("place_id")
        location_dict[index.get("name")]["rating"] = index.get("rating")
        location_dict[index.get("name")]["lat"] = index.get("geometry").get(
            "location").get("lat")
        location_dict[index.get("name")]["lng"] = index.get("geometry").get(
            "location").get("lng")
        location_dict[index.get("name")]["classification"] = index.get("types")
        location_dict[index.get("name")]["price_lvl"] = index.get(
            "price_level")
        location_dict[index.get("name")]["pull_address"] = index.get(
            "vicinity")
    return location_dict


def displayPlaces(dict_of_location: dict) -> None:
    """
    Given a list of location display it in a nicely readable format
    :param dict_of_location: A dictionary with all the locations nearby
    :type dict_of_location: dict
    :return: None
    :rtype: None
    """
    # Neatly Display The List Of Locations within 20 Minutes Of the Users Input Location And Transportation Method
    for location in dict_of_location:
        print(f"{location}:")
        for item in dict_of_location[location]:
            print(f"{item}:{dict_of_location[location][item]}")
        print(f'\n{"*":*^25}\n')


def addLocationAddressData(locations_dict: dict) -> None:
    """
    Given a dictionary of all location add their address to the dict
    :param locations_dict: A dictionary with all the locations nearby
    :type locations_dict: dict
    :return: None
    :rtype: None
    """
    # Convert the list of location to list of addresses - reverse_geocode()
    for location in locations_dict:
        locations_dict[location]["address"] = gomaps.reverse_geocode(
            latlng=locations_dict[location]["place_id"])[0].get(
                "formatted_address")


def computeTime(locations_dict: dict, origin: str, mode: str) -> None:
    """
    Given a dictionary of all location, the starting address, and mode of transportation compute the time it takes to get to each location and add to dict.
    :param locations_dict: A dictionary with all the locations nearby
    :type locations_dict: dict
    :param origin: The address of the starting location
    :type origin: str
    :param mode: The type of transportation
    :type mode: str
    :return: None
    :rtype: None
    """
    # Make a list of Matrix for each type of transportation - distance_matrix()
    for location in locations_dict:
        locations_dict[location]["travel_time"] = gomaps.distance_matrix(
            origins=origin,
            destinations=locations_dict[location].get("address"),
            mode=mode)["rows"][0]["elements"][0].get("duration",
                                                     {}).get("text", {})


def removeFarLocations(locations_dict):
    """
    Given a dictionary of all location remove any location that is more then 20 min away
    :param locations_dict: A dictionary with all the locations nearby
    :type locations_dict: dict
    :return: None
    :rtype: None
    """
    #Remove locations that were found further than 20 minutes from your specified location -
    for location in list(locations_dict.keys()):
        if "hours" in locations_dict[location][
                "travel_time"] or "hour" in locations_dict[location][
                    "travel_time"]:
            del locations_dict[location]
    for location in list(locations_dict.keys()):
        num = locations_dict[location]["travel_time"]
        if len(num) == 0 or int(num.split()[0]) > 20:
            del locations_dict[location]

# Main Of Applications:

#### Get the starting location from the user

In [ ]:
origin_location = collectInfo()

#### Get the type of transportation the user wants to use

In [ ]:
selection = collectTransportation()

#### Create the dictionary of all the location

In [ ]:
locations_dict = serechForPlaces(getGeoCode(origin_location), RADIUS[0])

#### Add all the location address to data dict

In [ ]:
addLocationAddressData(locations_dict)

#### Compute the travel time for all location in dict

In [ ]:
computeTime(locations_dict, origin_location, MODE_OF_TRANS[selection])

#### Remove locations that take longer then 20 min

In [ ]:
removeFarLocations(locations_dict)

#### Display those locations

In [ ]:
displayPlaces(locations_dict)